<a href="https://colab.research.google.com/github/huyqn-umich/myNRELResearchProject/blob/main/WRDB_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import requests
import pandas as pd
import urllib.parse
import time
import zipfile
import os
import glob

API_KEY = "mXHYEn2lt6ZWIMgZzAzQoCW1h8Q6vd8DvxT8AD04"
EMAIL = "huyqn180295@gmail.com"
BASE_URL = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/offshore-great-lakes-download.json?"
POINTS = [
'193879'
]

def main():
    input_data = {
        'attributes': 'friction_velocity_2m,inversemoninobukhovlength_2m,skin_temperature,roughness_length,surface_heat_flux,surface_sea_temperature,pressure_0m,pressure_100m,pressure_200m,pressure_300m,relativehumidity_2m,precipitationrate_0m,temperature_100m,turbulent_kinetic_energy_100m,winddirection_100m,windspeed_100m,temperature_10m,turbulent_kinetic_energy_10m,winddirection_10m,windspeed_10m,temperature_120m,turbulent_kinetic_energy_120m,winddirection_120m,windspeed_120m,temperature_140m,turbulent_kinetic_energy_140m,winddirection_140m,windspeed_140m,temperature_160m,turbulent_kinetic_energy_160m,winddirection_160m,windspeed_160m,temperature_180m,turbulent_kinetic_energy_180m,winddirection_180m,windspeed_180m,temperature_200m,turbulent_kinetic_energy_200m,winddirection_200m,windspeed_200m,temperature_20m,turbulent_kinetic_energy_20m,winddirection_20m,windspeed_20m,temperature_220m,turbulent_kinetic_energy_220m,winddirection_220m,windspeed_220m,temperature_240m,turbulent_kinetic_energy_240m,winddirection_240m,windspeed_240m,temperature_260m,turbulent_kinetic_energy_260m,winddirection_260m,windspeed_260m,temperature_280m,turbulent_kinetic_energy_280m,winddirection_280m,windspeed_280m,temperature_2m,temperature_300m,turbulent_kinetic_energy_300m,winddirection_300m,windspeed_300m,temperature_400m,turbulent_kinetic_energy_400m,winddirection_400m,windspeed_400m,temperature_40m,turbulent_kinetic_energy_40m,winddirection_40m,windspeed_40m,temperature_500m,turbulent_kinetic_energy_500m,winddirection_500m,windspeed_500m,temperature_60m,turbulent_kinetic_energy_60m,winddirection_60m,windspeed_60m,temperature_80m,turbulent_kinetic_energy_80m,winddirection_80m,windspeed_80m',
        'interval': '60',
        'to_utc': 'false',
        'include_leap_day': 'true',

        'api_key': API_KEY,
        'email': EMAIL,
    }
    for name in ['2020']:
        print(f"Processing name: {name}")
        for id, location_ids in enumerate(POINTS):
            input_data['names'] = [name]
            input_data['location_ids'] = location_ids
            print(f'Making request for point group {id + 1} of {len(POINTS)}...')

            if '.csv' in BASE_URL:
                url = BASE_URL + urllib.parse.urlencode(data, True)
                # Note: CSV format is only supported for single point requests
                # Suggest that you might append to a larger data frame
                data = pd.read_csv(url)
                print(f'Response data (you should replace this print statement with your processing): {data}')
                # You can use the following code to write it to a file
                # data.to_csv('SingleBigDataPoint.csv')
            else:
                headers = {
                  'x-api-key': API_KEY
                }
                data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
                download_url = data['outputs']['downloadUrl']
                # Display path to the .csv file
                csv_path = locate_csv_file(download_url)
                print(f'Successfully located the .csv file: {csv_path}')
                # Delay for 1 second to prevent rate limiting
                time.sleep(1)
            print(f'Processed')


def locate_csv_file(url):
    # Download the zip file
    zip_path = '/content/zip_folder.zip'
    response = requests.get(url)
    with open(zip_path, 'wb') as file:
        file.write(response.content)

    # Extract the zip file
    extract_path = '/content/extracted_folder'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Identify the CSV file path in the extracted folder
    csv_path = glob.glob(os.path.join(extract_path, '**/*.csv'))[0]
    return csv_path

def get_response_json_and_handle_errors(response: requests.Response) -> dict:
    """Takes the given response and handles any errors, along with providing
    the resulting json

    Parameters
    ----------
    response : requests.Response
        The response object

    Returns
    -------
    dict
        The resulting json
    """
    if response.status_code != 200:
        print(f"An error has occurred with the server or the request. The request response code/status: {response.status_code} {response.reason}")
        print(f"The response body: {response.text}")
        exit(1)

    try:
        response_json = response.json()
    except:
        print(f"The response couldn't be parsed as JSON, likely an issue with the server, here is the text: {response.text}")
        exit(1)

    if len(response_json['errors']) > 0:
        errors = '\n'.join(response_json['errors'])
        print(f"The request errored out, here are the errors: {errors}")
        exit(1)
    return response_json

if __name__ == "__main__":
    main()

Processing name: 2020
Making request for point group 1 of 1...
Successfully located the .csv file: /content/extracted_folder/5d2b1a8ffd9cd151c3e6994cc1200976/193879_42.30_-83.69_2020.csv
Processed
